In [1]:
%%capture
!pip install wandb

In [1]:
import wandb
from wandb.keras import WandbCallback

configs = {
              "learning_rate": 0.001,
              "epochs": 12,
              "batch_size": 16
           }

wandb.login()
run = wandb.init(project='face-recognition', config=configs)
config = wandb.config


wandb: Currently logged in as: zahra_zarrabi (use `wandb login --relogin` to force relogin)


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense,Conv2D,Flatten
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# **👩‍🍳 Prepare Dataset**

In [3]:
image_data_generator = ImageDataGenerator(rescale=1./255,
                                          horizontal_flip = True,
                                          validation_split= 0.2)

train_data = image_data_generator.flow_from_directory('/content/drive/MyDrive/7-7 dataset',
                                                      class_mode='categorical',
                                                      batch_size=config.batch_size,
                                                      shuffle=True,
                                                      target_size=(512, 512),
                                                      subset='training')

val_data = image_data_generator.flow_from_directory('/content/drive/MyDrive/7-7 dataset',
                                                      class_mode='categorical',
                                                      batch_size=config.batch_size,
                                                      shuffle=False,
                                                      target_size=(512, 512),
                                                      subset='validation')

Found 1091 images belonging to 14 classes.
Found 268 images belonging to 14 classes.


In [ ]:
train_images = next(train_data)[0]
plt.figure(figsize = (8,8))
for i in range(16):
  plt.subplot(4,4,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(train_images[i])

# 🧠 Define the Model and the Training Loop

In [5]:
class MyModel(tf.keras.Model):
  def __init__(self,number_of_classes):
    super().__init__()
    self.conv1=Conv2D(32,(3,3),input_shape=(512,512,3),activation='relu')
    self.conv1=Conv2D(16,(3,3),activation='relu')
    self.flatten=Flatten()
    self.fc1=Dense(128,activation='relu')
    self.fc2=Dense(number_of_classes, activation='softmax')
  def call(self,x):
  #   input = self.conv1(self.inputs)
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.fc1(x)
    output = self.fc2(x)
    return output

In [6]:
model = MyModel(14)

In [7]:
loss_function = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [8]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_acc = tf.keras.metrics.CategoricalAccuracy(name='train_acc')

val_loss = tf.keras.metrics.Mean(name='val_loss')
val_acc = tf.keras.metrics.CategoricalAccuracy(name='val_acc')


In [9]:
def train_step(images,y):
  with tf.GradientTape() as tape:
    y_pred = model(images,training=True)
    # print('y_pred',y_pred)
    # print('y', y.shape)
    loss = loss_function(y,y_pred)
  gradients = tape.gradient(loss,model.trainable_variables)
  optimizer.apply_gradients(zip(gradients,model.trainable_variables))
  train_loss(loss)
  # print('y', y)
  # print('y_pred', y_pred)
  train_acc(y,y_pred)
  # print('jnjnj')
 

In [10]:
def val_step(images,y):
  y_pred = model(images,training=False)
  loss = loss_function(y,y_pred)

  val_loss(loss)
  val_acc(y,y_pred)

In [11]:
def train():
  epochs=12
  for epoch in range(epochs):

    train_loss.reset_states()
    train_acc.reset_states()
    val_loss.reset_states()
    val_acc.reset_states()

    cnt = 0
    for cnt in tqdm(range(train_data.__len__())):
      images,labels = next(train_data)
      train_step(images,labels)
    
    for cnt in tqdm(range(val_data.__len__())):
      images,labels = next(val_data)
      val_step(images,labels)

    print('epoch:',epoch+1)
    print('loss:',train_loss.result())
    print('accuracy:',train_acc.result())
    print('val loss:',val_loss.result())
    print('val accuracy:',val_acc.result())
    print('-------------------------------------------------------------------')


    #log metrics using wandb.log
    wandb.log({'epochs': epoch,
                'loss': train_loss.result(),
                'acc': train_acc.result(), 
                'val_loss': val_loss.result(),
                'val_acc':val_acc.result()})

In [12]:
#fit
train()

100%|██████████| 17/17 [03:20<00:00, 11.79s/it]


epoch: 1
loss: tf.Tensor(64.53203, shape=(), dtype=float32)
accuracy: tf.Tensor(0.14023831, shape=(), dtype=float32)
val loss: tf.Tensor(2.8774166, shape=(), dtype=float32)
val accuracy: tf.Tensor(0.16044776, shape=(), dtype=float32)
-------------------------------------------------------------------


100%|██████████| 17/17 [00:06<00:00,  2.47it/s]


epoch: 2
loss: tf.Tensor(2.3759487, shape=(), dtype=float32)
accuracy: tf.Tensor(0.33638865, shape=(), dtype=float32)
val loss: tf.Tensor(2.645093, shape=(), dtype=float32)
val accuracy: tf.Tensor(0.2761194, shape=(), dtype=float32)
-------------------------------------------------------------------


100%|██████████| 17/17 [00:06<00:00,  2.50it/s]


...

epoch: 10
loss: tf.Tensor(0.028911216, shape=(), dtype=float32)
accuracy: tf.Tensor(0.9917507, shape=(), dtype=float32)
val loss: tf.Tensor(2.0054135, shape=(), dtype=float32)
val accuracy: tf.Tensor(0.5895522, shape=(), dtype=float32)
-------------------------------------------------------------------


100%|██████████| 17/17 [00:06<00:00,  2.45it/s]


epoch: 11
loss: tf.Tensor(0.024186462, shape=(), dtype=float32)
accuracy: tf.Tensor(0.99541706, shape=(), dtype=float32)
val loss: tf.Tensor(2.0855834, shape=(), dtype=float32)
val accuracy: tf.Tensor(0.6231343, shape=(), dtype=float32)
-------------------------------------------------------------------


100%|██████████| 17/17 [00:06<00:00,  2.46it/s]

epoch: 12
loss: tf.Tensor(0.0055853464, shape=(), dtype=float32)
accuracy: tf.Tensor(1.0, shape=(), dtype=float32)
val loss: tf.Tensor(1.9512138, shape=(), dtype=float32)
val accuracy: tf.Tensor(0.5895522, shape=(), dtype=float32)
-------------------------------------------------------------------


In [ ]:
model.save_weights('/content/drive/MyDrive/model1/')

In [ ]:
model.load_weights('/content/drive/MyDrive/model1/')